In [ ]:
from pathlib import Path
import sys
import cv2
import torch
import pandas as pd
import os
import contextlib
import tqdm

from detectron2.config import get_cfg
from detectron2.modeling import build_model
from detectron2.checkpoint import DetectionCheckpointer

sys.path.append("/home/shamil/AILab/Rosreest")

from utils.polygons.io import extract_polygons_from_coco
from utils.polygons.io import masks_to_polygons
from utils.polygons.metrics import instance_f1_score


In [2]:
dataset_folder = Path('/home/shamil/AILab/new_data/dataset/test/')
dst_file = Path('/home/shamil/AILab/new_data/dataset/test/metrics.csv')

config_file = Path('/home/shamil/AILab/Rosreest/config/mask_rcnn_R_50_FPN_3x.yaml')
weights_path = Path("/home/shamil/AILab/artificial_cartographer/artificial_cartographer"
                    "/model_weights/aerial_summer_pieceofland.pth")
metrics = 'f_score'
device = 'cpu'

assert dataset_folder.is_dir()
assert config_file.is_file()
assert weights_path.is_file()

assert metrics == 'f_score'

In [3]:
annotation_file = dataset_folder / 'uchastok_test2021.json'
assert annotation_file.is_file()

annotations = extract_polygons_from_coco(annotation_file)


In [3]:
annotations[1]

In [5]:
with open(os.devnull, 'w') as devnull:
    with contextlib.redirect_stdout(devnull):
        cfg = get_cfg()
        cfg.merge_from_file(config_file)
        cfg.MODEL.DEVICE = device

        model = build_model(cfg)

        DetectionCheckpointer(model).load(str(weights_path))
        model.eval()

In [2]:
result = pd.DataFrame({}, columns=['image_name', 'f_score'])

for image_id, meta in tqdm.tqdm(annotations.items(), total=385):

    image_name = meta['file_name']
    gt_polygons = meta['polygons']

    # print(f"Processing image_name={image_name}, image_id={image_id}")

    image_path = dataset_folder / 'uchastok_2021' / image_name
    image = cv2.imread(str(image_path))
    height, width = image.shape[:2]

    with torch.no_grad():
        image_np = image.astype("float32").transpose(2, 0, 1)
        image = torch.as_tensor(image_np, device=device)
        inputs = {'image': image, 'height': height, 'width': width}

        y = model.forward([inputs])
        instances = y[0]['instances'].to(device)
        masks = instances.pred_masks.numpy().astype('uint8')

    pred_polygons = masks_to_polygons(masks)

    f_score, _ = instance_f1_score(
        gt=gt_polygons,
        pred=pred_polygons,
        _format='vector',
        iou_threshold=0.5
    )

    result = result.append({"image_name": image_name, "f_score": f_score, 'gt': gt_polygons, "pred": pred_polygons}, ignore_index=True)

result[["image_name", "f_score"]].to_csv(dst_file)

In [1]:
result.head(10)


In [ ]:
print(f"Mean f_score: {result['f_score'].mean()}")
